In [ ]:
%%capture
import warnings
from datetime import date, timedelta
from google.colab import drive
!pip install seaborn
!pip install yahoo_fin
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import yfinance as yf
from tqdm import tqdm
import pandas as pd
from datetime import datetime
import holidays
import random
from google.colab import files
from statistics import mode
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
import concurrent.futures
!pip install gdown
import gdown
from scipy.stats import zscore
drive.mount('/content/drive')
import yahoo_fin.stock_info as si
# Non mostro la numerazione scientifica
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Mostro separatori delle migliai con la virgola
pd.options.display.float_format = '{:,.3f}'.format

cartella_csv = '/content/drive/MyDrive/Investimenti/finanza quant/'


In [ ]:
# Solo i tickers che ho in portafoglio

prtf = ['TRMD', 'MSFT', 'STLA', 'NVDA', 'MC.PA', 'LRCX', 'META', 'AMAT', 'CFRUY', 'URI', 'AAPL', 'DHI', 'CHRD', 'LPG'
,'SMCI','MOD']

symbol_list = prtf

all_tickers_filtered = prtf

all_tickers = prtf

print(len(prtf))

In [ ]:
from tqdm import tqdm

def fetch_fundamental_data(symbol):
    try:
        stock = yf.Ticker(symbol).info
        return stock
    except Exception as e:
        print(f"Ignorato errore per il simbolo {symbol}: {e}")
        return None

# Inizializza una lista per conservare i dati fondamentali
fundamental_data = []

# Utilizza tqdm per aggiungere una progress bar al ciclo
# for symbol in tqdm(all_tickers, desc="Fetching data"):
#     data = fetch_fundamental_data(symbol)
#     if data is not None:
#         fundamental_data.append(data)

# Crea il DataFrame finale dai dati fondamentali raccolti
df_fundamental = pd.DataFrame(fundamental_data)

In [ ]:
df_fundamental

In [ ]:
# df_filtered = df_fundamental[(df_fundamental['industry'] == 'Oil & Gas Midstream') & (df_fundamental['marketCap'] > 3000000000)][['shortName', 'marketCap', 'dividendYield']].assign(dividendYield=lambda x: x['dividendYield'] * 100).sort_values(by='dividendYield', ascending=False)
# df_filtered = df_fundamental[(df_fundamental['dividendYield'] > 0.05) & (df_fundamental['marketCap'] > 3000000000)  ][['shortName', 'marketCap', 'dividendYield', 'country']].assign(dividendYield=lambda x: x['dividendYield'] * 100).sort_values(by='dividendYield', ascending=False)

# Definisci la funzione che esegue la moltiplicazione
def multiply_columns(df):
    # df['dividendYield'] *= 1
    # df['ebitdaMargins'] *= 1
    # df['returnOnEquity'] *= 1
    #df['priceToHighRatio'] = df['currentPrice'] / df['fiftyTwoWeekHigh'] * 100
    df['mostRecentQuarter'] = pd.to_datetime(df['mostRecentQuarter'], unit='s').dt.date
    return df.drop(columns=['currentPrice', 'fiftyTwoWeekHigh'])  #non mi interessa portarmi dietro df['currentPrice'] / df['fiftyTwoWeekHigh']


# Filtra e seleziona il DataFrame, applica la funzione e ordina
df_filtered = (
    df_fundamental
    #  [
        #(df_fundamental['industry'] == 'Oil & Gas Midstream') &
    # (df_fundamental['marketCap'] > 2000000000)]
    [['shortName',  'industry','dividendYield', 'ebitdaMargins', 'recommendationKey','totalCashPerShare',
      'returnOnEquity', 'currentPrice', 'fiftyTwoWeekHigh','earningsQuarterlyGrowth','mostRecentQuarter'
      #'priceToHighRatio'
      ]]
    .pipe(multiply_columns)
    .sort_values(by='dividendYield', ascending=False)
)

#df_filtered = df_fundamental[(df_fundamental['industry'] == 'Oil & Gas Midstream') & (df_fundamental['marketCap'] > 2000000000)][['shortName', 'marketCap','52WeekChange','dividendYield','ebitdaMargins','recommendationKey']].assign(dividendYield=lambda x: x[['dividendYield','ebitdaMargins']] * 100).sort_values(by='dividendYield', ascending=False)
df_filtered
nuovo = df_filtered
nuovo

In [ ]:
file = "/content/drive/MyDrive/Investimenti/finanza quant/simple_kpis.csv"

In [ ]:
esistente = pd.read_csv(file, usecols=range(1, len(pd.read_csv(file, nrows=1).columns)))

In [ ]:
esistente['mostRecentQuarter'] = pd.to_datetime(esistente['mostRecentQuarter'])
nuovo['mostRecentQuarter'] = pd.to_datetime(nuovo['mostRecentQuarter'])

esistente['shortName'] = esistente['shortName'].astype(str)
nuovo['shortName'] = nuovo['shortName'].astype(str)

esistente['industry'] = esistente['industry'].astype(str)
nuovo['industry'] = nuovo['industry'].astype(str)


In [ ]:
appeso = pd.concat([esistente,nuovo]).drop_duplicates(subset=["shortName","mostRecentQuarter"]).sort_values(by=['shortName', 'mostRecentQuarter'])


In [ ]:
appeso.to_csv(file)
